In [1]:
val url = "http://shot.holycross.edu/ada-vases.tsv"

url: String = "http://shot.holycross.edu/ada-vases.tsv"

In [2]:
import scala.io.Source
val lines = Source.fromURL(url).getLines.toVector
lines.size

import scala.io.Source

lines: Vector[String] = Vector(
  "URI\tVase Number\tFabric\tTechnique\tSub Technique\tShape Name\tProvenance\tDate\tInscriptions\t Attributed To\t Decoration\t Collection Record\t Publication Record\t Inscriptions II\tMeasurements\tVolume\tWeight\tRestorations\tOrnament\t Technical Observations\tPleiades URI\tLatitude\tLongitude",
  "<a href=\"http://www.beazley.ox.ac.uk/record/8C5C25CA-9E1C-41A2-886A-2558611DCA04\" target=\"_blank\">http://www.beazley.ox.ac.uk/record/8C5C25CA-9E1C-41A2-886A-2558611DCA04</a>\t1\tATHENIAN\tBLACK-FIGURE\t\tAMPHORA, NECK\tITALY, ETRURIA, VULCI\t-550 to -500\t\tCHIUSI P by KUNZE-GOTTE | LEAGROS GROUP by KUNZE-GOTTE\tA,B: TRIPTOLEMOS ON WINGED CHARIOT BETWEEN DEMETER AND PERSEPHONE\tMunich, Antikensammlungen: J543 | Rome, Candelori Collection | Munich, Antikensammlungen: 1539\tCorpus Vasorum Antiquorum: MUNICH, MUSEUM ANTIKER KLEINKUNST 8, 56-57, BEILAGE D4, PLS.(1816,1820,1830) 398.4,402.1-2,412.2 <a target=_blank href=browseCVARec

In [3]:
val byColumns = lines.map(ln => ln.split("\t").toVector)
byColumns.size

byColumns: Vector[Vector[String]] = Vector(
  Vector(
    "URI",
    "Vase Number",
    "Fabric",
    "Technique",
    "Sub Technique",
    "Shape Name",
    "Provenance",
    "Date",
    "Inscriptions",
    " Attributed To",
    " Decoration",
    " Collection Record",
    " Publication Record",
    " Inscriptions II",
    "Measurements",
    "Volume",
    "Weight",
    "Restorations",
    "Ornament",
    " Technical Observations",
    "Pleiades URI",
    "Latitude",
    "Longitude"
  ),
  Vector(
    "<a href=\"http://www.beazley.ox.ac.uk/record/8C5C25CA-9E1C-41A2-886A-2558611DCA04\" target=\"_blank\">http://www.beazley.ox.ac.uk/record/8C5C25CA-9E1C-41A2-886A-2558611DCA04</a>",
    "1",
    "ATHENIAN",
    "BLACK-FIGURE",
    "",
    "AMPHORA, NECK",
    "ITALY, ETRURIA, VULCI",
    "-550 to -500",
    "",
    "CHIUSI P by KUNZE-GOTTE | LEAGROS GROUP by KUNZE-GOTTE",
...
res2_1: Int = 117751

In [4]:
val descriptions = byColumns.filter(row => row.size == 23).map( v => v(10))

descriptions: Vector[String] = Vector(
  " Decoration",
  "A,B: TRIPTOLEMOS ON WINGED CHARIOT BETWEEN DEMETER AND PERSEPHONE",
  "Under handle: VOLUTE KRATER",
  "A,B: TRIPTOLEMOS AND DEMETER",
  "A: ATHENA",
  "Body: CHILD AND HARE",
  "A: ATHENA, BETWEEN COLUMNS, SHIELD DEVICE, DISCS | B: MUSIC, YOUTH ON PLATFORM PLAYING PIPES BETWEEN JUDGES, SEATED",
  "WOMAN AND SATYR",
  "A: GIGANTOMACHY (?) HERAKLES AND ATHENA IN CHARIOT, WARRIOR, NONSENSE INSCRIPTIONS | B: HERAKLES PLAYING KITHARA AT ALTAR, IOLAOS, WOMAN, ATHENA, SHIELD DEVICE, SNAKE, NONSENSE INSCRIPTIONS | Under foot: GRAFFITO: MAN, SWASTIKA",
  "A: MAN WITH PHIALE AND STAFF, WOMAN WITH OINOCHOE | B: WOMAN",
  "A: SWAN BETWEEN LIONS | B: SIRENS | Neck A: SIREN | Neck B: LION",
  "A: DIONYSOS, SATYRS, ONE WITH LYRE AND MAENAD WITH PIPES, IN SHIP | B: DIONYSOS, MAENADS AND SATYRS IN SHIP",
  "Body: SPHINX, LIONS",
  "Body: WEDDED PAIR IN CHARIOT, WOMEN, YOUTHS WITH HYDRIA AND KITHARA, OLD MAN SEATED | Predella: PANTHERS, DEER | 

In [5]:
val tidier = descriptions.map(t => t.toLowerCase).map(descr => descr.replaceAll("[^a-z ]", ""))

tidier: Vector[String] = Vector(
  " decoration",
  "ab triptolemos on winged chariot between demeter and persephone",
  "under handle volute krater",
  "ab triptolemos and demeter",
  "a athena",
  "body child and hare",
  "a athena between columns shield device discs  b music youth on platform playing pipes between judges seated",
  "woman and satyr",
  "a gigantomachy  herakles and athena in chariot warrior nonsense inscriptions  b herakles playing kithara at altar iolaos woman athena shield device snake nonsense inscriptions  under foot graffito man swastika",
  "a man with phiale and staff woman with oinochoe  b woman",
  "a swan between lions  b sirens  neck a siren  neck b lion",
  "a dionysos satyrs one with lyre and maenad with pipes in ship  b dionysos maenads and satyrs in ship",
  "body sphinx lions",
  "body wedded pair in chariot women youths with hydria and kithara old man seated  predella panthers deer  shoulder theseus in nebris and minotaur between onlookers youths so

In [6]:
val k = 10
val iterations = 20
val stopWords = Array("the", "a", "and", "with", "holding", "for", "into", "between", "among", "some", "from")
val vocabSize = 10000
val minimumTokenLength = 3
val termsToDisplay = 15

k: Int = 10
iterations: Int = 20
stopWords: Array[String] = Array(
  "the",
  "a",
  "and",
  "with",
  "holding",
  "for",
  "into",
  "between",
  "among",
  "some",
  "from"
)
vocabSize: Int = 10000
minimumTokenLength: Int = 3
termsToDisplay: Int = 15

In [7]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import org.apache.spark.sql._
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.5/spark-sql_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.8.2/almond-spark_2.12-0.8.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.5/spark-parent_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.5.5/orc-mapreduce-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-hadoop/1.10.1/parquet-hadoop-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-vector/0.10.0/arrow-vector-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.8/scala-library-2.12.8.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.5/spark-catalyst_2.12-2.4.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sketch_

Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-jvm/3.1.5/metrics-jvm-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.6.7.1/jackson-module-scala_2.12-2.6.7.1.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-graphite/3.1.5/metrics-graphite-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-client/2.6.5/hadoop-client-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/commons-codec/commons-codec/1.10/commons-codec-1.10.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/janino/3.0.9/janino-3.0.9.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-format/2.4.0/parquet-format-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/com/ning/compress-lzf/1.0.3/compress-lzf-1.0.3.pom
Downloaded https://repo1.maven.org/maven2/com/cle

Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.1.0/scala-parser-combinators_2.12-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet/2.22.2/jersey-container-servlet-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/commons-net/commons-net/3.1/commons-net-3.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-shims/1.5.5/orc-shims-1.5.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-jackson/1.10.1/parquet-jackson-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.20.v20190813/jetty-server-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/3.1.5/metrics-core-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/javax/activation/activation/1.1.1/activation-1

Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/40/commons-parent-40.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.6.1/jackson-parent-2.6.1.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.16/slf4j-parent-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/35/commons-parent-35.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/34/commons-parent-34.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/23/commons-parent-23.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/17/commons-parent-17.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/17/apache-17.pom
Downloaded https://repo1.maven.org/maven2/org/apache/avro/avro-toplevel/1.8.2/avro-toplevel-1.8.2.pom
Downloaded https://repo1.maven.org/maven2/net/java/jvnet-parent/4/jvnet-parent-4.pom
Downloaded https://repo1.

Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/osgi-resource-locator/1.0.1/osgi-resource-locator-1.0.1.pom
Downloaded https://repo1.maven.org/maven2/javax/ws/rs/javax.ws.rs-api/2.0.1/javax.ws.rs-api-2.0.1.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.6.7/jackson-annotations-2.6.7.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-annotations/2.6.5/hadoop-annotations-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-http/9.4.20.v20190813/jetty-http-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-api/2.6.5/hadoop-yarn-api-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-app/2.6.5/hadoop-mapreduce-client-app-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/curator/curator-framework/2.6.0/curator-framework-2.6.0.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bun

...

Downloaded https://repo1.maven.org/maven2/com/google/inject/guice/3.0/guice-3.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-client/2.6.5/hadoop-client-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-common/2.6.5/hadoop-yarn-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-common/2.6.5/hadoop-mapreduce-client-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-server-common/2.6.5/hadoop-yarn-server-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/inject/javax.inject/1/javax.inject-1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-app/2.6.5/hadoop-mapreduce-client-app-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-client/2.6.5/hadoop-yarn-client-2.6.5-sources.jar
Downloaded https://repo1.maven.or

Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.16-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.0.6/scala-xml_2.12-1.0.6-sources.jar
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-core_2.12/3.5.3/json4s-core_2.12-3.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/external/javax.inject/2.4.0-b34/javax.inject-2.4.0-b34-sources.jar
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-jackson_2.12/3.5.3/json4s-jackson_2.12-3.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/ws/rs/javax.ws.rs-api/2.0.1/javax.ws.rs-api-2.0.1-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/annotation/javax.annotation-api/1.2/javax.annotation-api-1.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bundles/repackaged/jersey-guava/2.22.2/jersey-guava-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/h

Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.6.7.1/jackson-module-scala_2.12-2.6.7.1-sources.jar
Downloaded https://repo1.maven.org/maven2/io/airlift/aircompressor/0.10/aircompressor-0.10-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/2.4.5/spark-core_2.12-2.4.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-vector/0.10.0/arrow-vector-0.10.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/ivy/ivy/2.4.0/ivy-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.10.1/parquet-column-1.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/net/sf/py4j/py4j/0.10.7/py4j-0.10.7-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.5/spark-sql_2.12-2.4.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.5/spark-catalyst_2.12-2.4.5-

Downloaded https://repo1.maven.org/maven2/net/sf/opencsv/opencsv/2.3/opencsv-2.3.pom
Downloaded https://repo1.maven.org/maven2/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.12/0.13.0/spire_2.12-0.13.0.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/core/1.1.2/core-1.1.2.pom
Downloaded https://repo1.maven.org/maven2/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-macros_2.12/0.13.2/breeze-macros_2.12-0.13.2.pom
Downloaded https://repo1.maven.org/maven2/com/google/code/findbugs/jsr305/1.3.9/jsr305-1.3.9.pom
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/parent/1.1/parent-1.1.pom
Downloaded https://repo1.maven.org/maven2/org/typelevel/machinist_2.12/0.6.1/machinist_2.12-0.6.1.pom
Downloaded https://re

import $ivy.$                                  

import org.apache.spark.sql._

import $ivy.$                                    

In [8]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [9]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2-sources.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.jar


Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@2dac7368

In [10]:
import org.apache.spark.ml.clustering.LDA
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.mllib.linalg.Vector
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.types.IntegerType
import org.apache.spark.sql.functions._

import org.apache.spark.ml.clustering.LDA

import org.apache.spark.ml.feature.RegexTokenizer

import org.apache.spark.ml.feature.StopWordsRemover

import org.apache.spark.ml.feature.CountVectorizer

import org.apache.spark.mllib.linalg.Vector

import scala.collection.mutable.WrappedArray

import org.apache.spark.sql.types.IntegerType

import org.apache.spark.sql.functions._

In [11]:
val txtRdd = spark.sparkContext.parallelize(tidier).zipWithIndex

zipWithIndex at cmd10.sc:1

1 / 1

txtRdd: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[1] at zipWithIndex at cmd10.sc:1

In [12]:
import spark.sqlContext.implicits._

val corpus_df = txtRdd.toDF("corpus", "id")

import spark.sqlContext.implicits._


corpus_df: DataFrame = [corpus: string, id: bigint]

In [13]:
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        val str = cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
        if (truncate > 0 && str.length > truncate) {
          if (truncate < 4) str.substring(0, truncate)
          else str.substring(0, truncate - 3) + "..."
        } else {
          str
        }
      }: Seq[String]
    }

    publish.html(s"""
      <table class="table">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

defined class RichDF

In [14]:
val tokenizer = new RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(minimumTokenLength).setInputCol("corpus").setOutputCol("tokens")
val tokenized_df = tokenizer.transform(corpus_df)

tokenizer: RegexTokenizer = regexTok_0def35ce9b62
tokenized_df: DataFrame = [corpus: string, id: bigint ... 1 more field]

In [15]:
val remover = new StopWordsRemover().setStopWords(stopWords).setInputCol("tokens").setOutputCol("filtered")
val filtered_df = remover.transform(tokenized_df)

remover: StopWordsRemover = stopWords_96681db7a813
filtered_df: DataFrame = [corpus: string, id: bigint ... 2 more fields]

In [16]:
val vectorizer = new CountVectorizer().setInputCol("filtered").setOutputCol("features").setVocabSize(vocabSize).setMinDF(5).fit(filtered_df)
val countVectors = vectorizer.transform(filtered_df).select("id", "features")

flatMap at CountVectorizer.scala:205

2 / 2

count at CountVectorizer.scala:230

2 / 2

top at CountVectorizer.scala:233

2 / 2

vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_e5038a7077ac
countVectors: DataFrame = [id: bigint, features: vector]

In [17]:
val lda = new LDA().setK(k).setMaxIter(iterations)
val model = lda.fit(countVectors)

count at LDAOptimizer.scala:419

2 / 2

first at LDAOptimizer.scala:420

1 / 1

isEmpty at LDAOptimizer.scala:450

...

isEmpty at LDAOptimizer.scala:450

1 / 1

treeAggregate at LDAOptimizer.scala:504

2 / 2

lda: LDA = lda_cb1086bb96f3
model: org.apache.spark.ml.clustering.LDAModel = lda_cb1086bb96f3

In [18]:
val topics = model.describeTopics(termsToDisplay)

topics: DataFrame = [topic: int, termIndices: array<int> ... 1 more field]

In [19]:
topics.showHTML(truncate=1000)

topic,termIndices,termWeights
0,"[2, 5, 4, 0, 29, 1, 9, 52, 64, 69, 7, 80, 56, 10, 26]","[0.16003012977631637, 0.08287610273613379, 0.04944689270215101, 0.03818290851696983, 0.032611724412986584, 0.02695560204384769, 0.025848372856060075, 0.025097931535173635, 0.023652104182011892, 0.02082896843156177, 0.016520089049119772, 0.015564667274861797, 0.015548655982932855, 0.013340814982153771, 0.013302374694814814]"
1,"[203, 215, 76, 328, 112, 326, 6, 1, 31, 400, 551, 148, 138, 626, 124]","[0.05859928020513258, 0.036479785446466934, 0.03145130534516669, 0.030255344480794824, 0.025485730091209077, 0.02347767316002924, 0.020533395035346184, 0.017858509560524212, 0.017787038725351215, 0.017458550595419085, 0.011941539945256877, 0.010371150798579141, 0.008144671256640278, 0.005654584092334529, 0.005341121281590642]"
2,"[40, 15, 13, 78, 108, 0, 104, 14, 171, 26, 175, 158, 138, 9, 58]","[0.10994347547894566, 0.08143868571105786, 0.07065510791681028, 0.06287117410979673, 0.046769272095362635, 0.03233465259908366, 0.019206185661552643, 0.017068004607801824, 0.016301906681950844, 0.015473655661155114, 0.014102310493782964, 0.013826335100834562, 0.01208554590909783, 0.011923324019538259, 0.011702412120666672]"
3,"[1, 4, 0, 6, 5, 2, 16, 30, 10, 7, 47, 3, 45, 75, 57]","[0.12161473908366859, 0.09652855502874602, 0.07439713916242284, 0.04679484061215971, 0.04337412694067602, 0.038887931703771694, 0.028869827906194623, 0.027814689296532785, 0.02357421289034284, 0.02180310029021482, 0.020617052517255936, 0.016860260929703635, 0.016706441625372725, 0.013717069087626274, 0.011390095354000719]"
4,"[0, 14, 27, 34, 53, 51, 59, 31, 22, 90, 86, 82, 77, 68, 49]","[0.10021360183136394, 0.07761598424770023, 0.05212792618532028, 0.0405347444847571, 0.03440355520716408, 0.03396222451055591, 0.03267472755311903, 0.029251400858681983, 0.02651919660687892, 0.021549470150573936, 0.019970896824353067, 0.01771739407573983, 0.01663734818508818, 0.016361268360528713, 0.01342346604282037]"
5,"[1, 12, 9, 5, 50, 0, 46, 4, 58, 60, 10, 67, 73, 39, 96]","[0.06442344230205073, 0.05803614232162765, 0.05488128460704827, 0.03602447941698823, 0.028637399268721036, 0.026743749205295313, 0.026419035644824746, 0.02450942299852522, 0.019600319091237658, 0.017363297452125892, 0.01708337653864012, 0.01660927674139233, 0.015261103408282389, 0.015177911869736051, 0.01471030516653282]"
6,"[19, 23, 24, 28, 41, 3, 15, 0, 17, 38, 102, 106, 12, 116, 18]","[0.05218738167737433, 0.050205901119064256, 0.046700901816735134, 0.036625250693974314, 0.03239199852738959, 0.03043064186315965, 0.029539224941270405, 0.02836174467574464, 0.024822768030964944, 0.021536576361313082, 0.017107177543117504, 0.01452510307934931, 0.013106869398274866, 0.01303052740048003, 0.011109427983618966]"
7,"[236, 262, 379, 402, 547, 641, 183, 601, 582, 438, 636, 496, 307, 433, 8]","[0.03564516414288579, 0.032386389236760266, 0.007817929660358724, 0.007779137165490934, 0.007112612695861381, 0.0066633980109445665, 0.006436271442445303, 0.006417894402852797, 0.006304816520055969, 0.006285046698327117, 0.0059464806428473635, 0.005823420120116743, 0.00541587716329412, 0.005397797495618095, 0.005283555677289462]"
8,"[17, 156, 180, 163, 142, 2, 204, 198, 88, 28, 87, 147, 168, 0, 21]","[0.08311197525209028, 0.02412659217285607, 0.021661348746752754, 0.020784932973300887, 0.01930705193294678, 0.018392796591881186, 0.016485140027434286, 0.01560391243786319, 0.015570838097186401, 0.015259738706241265, 0.014719106095834425, 0.014606815055288923, 0.01301985440709138, 0.012820540824409055, 0.012091870420078095]"
9,"[3, 7, 0, 8, 11, 1, 20, 6, 13, 25, 2, 42, 18, 21, 32]","[0.08400846140679658, 0.043838920814401536, 0.03536999897756521, 0.033967623081002644, 0.03347737399568479, 0.02960477463556327, 0.0284233966811081, 0.027195748498439275, 0.025309007583680987, 0.02111378522466367, 0.02080430605743521, 0.018035788454942414, 0.017749962123476255, 0.017127506364081977, 0.016369212354609846]"


In [20]:
val topicLabels = topics.select("termIndices").map { case Row(r:  WrappedArray[Integer]) => r.map( i => vectorizer.vocabulary(i) ) }
val labelsNumberedLong = topicLabels.rdd.zipWithIndex.toDF("terms", "topicLong")
val labelsIndexed = labelsNumberedLong.withColumn("topic", $"topicLong".cast(IntegerType)).drop("topicLong")

val topicsWithTerms = labelsIndexed.join(topics, labelsIndexed.col("topic") === topics.col("topic")).drop(labelsIndexed.col("topic"))

zipWithIndex at cmd19.sc:2

1 / 1

topicLabels: Dataset[WrappedArray[String]] = [value: array<string>]
labelsNumberedLong: DataFrame = [terms: array<string>, topicLong: bigint]
labelsIndexed: DataFrame = [terms: array<string>, topic: int]
topicsWithTerms: DataFrame = [terms: array<string>, topic: int ... 2 more fields]

In [21]:
val weightedLabels = topicsWithTerms.withColumn("termsWithWeight", expr("zip_with(terms, termWeights, (t,w) -> concat(t, ' ', w))"))

weightedLabels: DataFrame = [terms: array<string>, topic: int ... 3 more fields]

In [22]:
weightedLabels.select("topic", "termsWithWeight").showHTML(truncate=1000)

run at ThreadPoolExecutor.java:1149

2 / 2

take at cmd12.sc:4

1 / 1

take at cmd12.sc:4

1 / 1

topic,termsWithWeight
0,"[woman 0.16003012977631637, man 0.08287610273613379, youth 0.04944689270215101, body 0.03818290851696983, komos 0.032611724412986584, draped 0.02695560204384769, figure 0.025848372856060075, nike 0.025097931535173635, altar 0.023652104182011892, column 0.02082896843156177, seated 0.016520089049119772, winged 0.015564667274861797, phiale 0.015548655982932855, staff 0.013340814982153771, running 0.013302374694814814]"
1,"[griffin 0.05859928020513258, heads 0.036479785446466934, lid 0.03145130534516669, grypomachy 0.030255344480794824, fig 0.025485730091209077, arimasp 0.02347767316002924, youths 0.020533395035346184, draped 0.017858509560524212, head 0.017787038725351215, griffins 0.017458550595419085, arimasps 0.011941539945256877, diskos 0.010371150798579141, horseback 0.008144671256640278, pig 0.005654584092334529, sword 0.005341121281590642]"
2,"[palmettes 0.10994347547894566, warrior 0.08143868571105786, satyr 0.07065510791681028, horseman 0.06287117410979673, amazon 0.046769272095362635, body 0.03233465259908366, amazonomachy 0.019206185661552643, floral 0.017068004607801824, nonsense 0.016301906681950844, running 0.015473655661155114, oriental 0.014102310493782964, horses 0.013826335100834562, horseback 0.01208554590909783, figure 0.011923324019538259, horse 0.011702412120666672]"
3,"[draped 0.12161473908366859, youth 0.09652855502874602, body 0.07439713916242284, youths 0.04679484061215971, man 0.04337412694067602, woman 0.038887931703771694, men 0.028869827906194623, symposium 0.027814689296532785, staff 0.02357421289034284, seated 0.02180310029021482, reclining 0.020617052517255936, one 0.016860260929703635, spears 0.016706441625372725, funerary 0.013717069087626274, departing 0.011390095354000719]"
4,"[body 0.10021360183136394, floral 0.07761598424770023, palmette 0.05212792618532028, animal 0.0405347444847571, chain 0.03440355520716408, frieze 0.03396222451055591, lotus 0.03267472755311903, head 0.029251400858681983, ivy 0.02651919660687892, bud 0.021549470150573936, siren 0.019970896824353067, panther 0.01771739407573983, sphinx 0.01663734818508818, deer 0.016361268360528713, wreath 0.01342346604282037]"
5,"[draped 0.06442344230205073, chariot 0.05803614232162765, figure 0.05488128460704827, man 0.03602447941698823, athlete 0.028637399268721036, body 0.026743749205295313, athletes 0.026419035644824746, youth 0.02450942299852522, horse 0.019600319091237658, handle 0.017363297452125892, staff 0.01708337653864012, apollo 0.01660927674139233, eyes 0.015261103408282389, hermes 0.015177911869736051, mounting 0.01471030516653282]"
6,"[herakles 0.05218738167737433, warriors 0.050205901119064256, shield 0.046700901816735134, device 0.036625250693974314, fight 0.03239199852738959, one 0.03043064186315965, warrior 0.029539224941270405, body 0.02836174467574464, athena 0.024822768030964944, lion 0.021536576361313082, devices 0.017107177543117504, falling 0.01452510307934931, chariot 0.013106869398274866, boeotian 0.01303052740048003, suspended 0.011109427983618966]"
7,"[around 0.03564516414288579, zone 0.032386389236760266, incense 0.007817929660358724, burner 0.007779137165490934, prokrustes 0.007112612695861381, sinis 0.0066633980109445665, ram 0.006436271442445303, deeds 0.006417894402852797, skiron 0.006304816520055969, sea 0.006285046698327117, pelias 0.0059464806428473635, tongues 0.005823420120116743, trees 0.00541587716329412, agriculture 0.005397797495618095, women 0.005283555677289462]"
8,"[athena 0.08311197525209028, owl 0.02412659217285607, olive 0.021661348746752754, columns 0.020784932973300887, sprig 0.01930705193294678, woman 0.018392796591881186, surmounted 0.016485140027434286, sprigs 0.01560391243786319, tree 0.015570838097186401, device 0.015259738706241265, neck 0.014719106095834425, achilles 0.014606815055288923, cocks 0.01301985440709138, body 0.012820540824409055, playing 0.012091870420078095]"
9,"[one 0.08400846140679658, seated 0.043838920814401536, body 0.03536999897756521, wo